In [ ]:
# Imports
import time
import math
import requests
import random
import dateutil.parser as dateparse
import datetime
import itertools
from typing import Callable

# Parameters

JWT: Get it from tournesol.app

- open website, open dev tools, get any request to Tournesol api, see Request Headers, get `Authentication="Bearer ..."` value
- DO NOT SHARE THIS TOKEN TO ANYONE. NEVER. IN ANY CONDITIONS. Even support will never need it.
- This token expires after some time of inactivity. If tool fails, try to update the token first.

LNGS: Pick all the languages of the videos to keep

In [ ]:
# PARAMETERS
JWT='Bearer xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
LNGS=['fr', 'en'] # Consolidation phase will only suggest videos in these languages


# Technical

Below is some technical stuff used for the suggestion mechanism.

Play all the next cells one by one, you do not need to change anything.

In [ ]:
# Functions utils
LAST_TNSL_CALL=datetime.datetime.utcnow()
def callTournesol(path: str):
	global LAST_TNSL_CALL
	BASE_URL='https://api.tournesol.app/'
	wait=1-(datetime.datetime.utcnow()-LAST_TNSL_CALL).total_seconds()
	if wait > 0:
		time.sleep(wait)
	response = requests.get(BASE_URL + path, headers={
		'Authorization': JWT
	})
	LAST_TNSL_CALL = datetime.datetime.utcnow()
	return response.json()

def callTournesolMulti(path: str, args:str=None, start:int=0, end:int=0):
	LIMIT=1000
	URL=f'{path}?limit={LIMIT}' + (('&' + args) if args else '')
	offset=start
	rs = callTournesol(URL + f'&offset={offset}')
	if not 'count' in rs or not 'results' in rs:
		print('##### ERROR #####')
		print(URL)
		print(rs)
		exit(-1)
		return []

	total=rs['count']
	allRes = rs['results']
	print(f'{len(allRes)}/{total}', end=' ')

	while len(allRes) < total and (end <= 0 or offset < end):
		offset += LIMIT
		rs = callTournesol(URL + f'&offset={offset}')
		total=rs['count']
		allRes += rs['results']
		print(f'{len(allRes)}', end=' ')
	print()

	return allRes

def get(json, default, *fields):
	for f in fields:
		if f in json:
			json = json[f]
			if not json:
				return default
		else:
			return default
	return json

def rndAB():
	return ('A', 'B') if random.random() > 0.5 else ('B', 'A')


def get_individual_score(vdata):
	arr = [s for s in get(vdata, [], 'individual_rating', 'criteria_scores') if s['criteria'] == 'largely_recommended']
	return arr[0]['score'] if arr else None


In [ ]:
# Import video lists
VData = dict

def crossmerge_lists(l1, l2): 
	minln = min(len(l1), len(l2))
	merged = [None]*(minln*2)
	merged[::2] = l2[:minln]
	merged[1::2] = l1[:minln]
	merged += l2[minln:] + l1[minln:]
	return merged

def get_lists():
	# Get
	print('Extracting compared videos...', end=' ')
	allRes:list[VData] = callTournesolMulti('users/me/contributor_ratings/videos', 'order_by=last_compared_at')

	print('Extracting rate_later list...', end=' ')
	allRes += callTournesolMulti('users/me/rate_later/videos')

	# Exclusions
	# Exclude videos when language not in LNGS & if has been compared within last 24h
	now = datetime.datetime.utcnow()
	allRes = [v for v in allRes if
		get(v, LNGS[0], 'entity', 'metadata', 'language') in LNGS
		and (now - dateparse.parse(get(v, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at'), ignoretz=True)).total_seconds() > (60*60*24)
	]

	# Collect
	cmp_vids:list[dict[str,VData]] = [dict(), dict(), dict(), dict()] # min 4 to not explode
	for v in allRes:
		cmps = get(v, 0, 'individual_rating', 'n_comparisons')
		while cmps >= len(cmp_vids):
			cmp_vids.append(dict())
		cmp_vids[cmps][v['entity']['uid']] = v

	for i in range(len(cmp_vids)):
		print(f"\t- {i} cmps:", len(cmp_vids[i]))

	# rate_later
	v_0 = sorted(cmp_vids[0].values(), key=lambda v:(
		#	Well noted from few contributors => Go first
		-get(v, 0, 'collective_rating', 'tournesol_score') / (get(v, None, 'collective_rating', 'n_contributors') if get(v, 0, 'collective_rating', 'n_contributors') > 1 else 2),
		#	When was added to rate later: older first
		 get(v, '9999-12-31T23:59:59', 'rate_later_metadata', 'created_at'),
		#	Publication date: older first
		 get(v, '2000-01-01T00:00:00', 'entity', 'metadata', 'publication_date'),
	))
	v_1 = sorted(cmp_vids[1].values(), key=lambda v:(
		#	Well noted from few contributors => Go first
		-get(v, 0, 'collective_rating', 'tournesol_score') / (get(v, None, 'collective_rating', 'n_contributors') if get(v, 0, 'collective_rating', 'n_contributors') > 1 else 2),
		#	Oldest last comparison first
		 get(v, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at'),
		#	Publication date: older first
		 get(v, '2000-01-01T00:00:00', 'entity', 'metadata', 'publication_date'),
	))
	# Put first from v_1 first, then alternate between v_0 & v_1 in order until one of them is exhausted, then push the rest
	rate_later = crossmerge_lists(v_0, v_1)

	# low_comparisons
	v_2 = sorted(cmp_vids[2].values(), key=lambda v:(
		#	oldest last comparison first
		get(v, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at'),
		#	Publication date: older first
		get(v, '2000-01-01T00:00:00', 'entity', 'metadata', 'publication_date'),
	))
	v_3 = sorted(cmp_vids[3].values(), key=lambda v:(
		#	oldest last comparison first
		get(v, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at'),
		#	Publication date: older first
		get(v, '2000-01-01T00:00:00', 'entity', 'metadata', 'publication_date'),
	))
	low_comparisons = crossmerge_lists(v_2, v_3)

	# well_compared
	v_4_p = itertools.chain.from_iterable(cmp_vids[i].values() for i in range(4, len(cmp_vids)))
	well_compared = sorted(v_4_p, key=lambda v:(
		#	oldest last comparison first
		get(v, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at'),
		#	Publication date: older first
		get(v, '2000-01-01T00:00:00', 'entity', 'metadata', 'publication_date'),
	))

	return (rate_later, low_comparisons, well_compared)

In [ ]:
# Cache DistComparisonare_sharing_comparisonser
class DistComparisonChecker:
	def __init__(self):
		self.cache: dict[str,set[str]] = dict()
		self.toupdate: set[str] = set()

	def _get_data_cached(self, vid):
		if vid in self.toupdate or vid not in self.cache:
			print(f'Obtaining comparisons with {vid}...', end=' ')
			allRes = callTournesolMulti(f"users/me/comparisons/videos/{vid}/")
			self.cache[vid] = set()
			for g in allRes:
				vid2 = g['entity_b' if g['entity_a']['uid'] == vid else 'entity_a']['uid']
				self.cache[vid].add(vid2)
				if not vid2 in self.cache:
					self.cache[vid2] = set()
					self.toupdate.add(vid2)
				self.cache[vid2].add(vid)
			if vid in self.toupdate:
				self.toupdate.remove(vid)
		return self.cache[vid]
	
	def are_sharing_comparisons(self, vdata1, vdata2):
		vid1 = vdata1['entity']['uid']
		vid2 = vdata2['entity']['uid']
		c1 = self._get_data_cached(vid1)
		c2 = self._get_data_cached(vid2)
		return (vid1 in c2) or (vid2 in c1) or (not c1.isdisjoint(c2))
	
	def have_cached_comparisons_shared(self, vdata1, vdata2):
		vid1 = vdata1['entity']['uid']
		vid2 = vdata2['entity']['uid']
		c1:set[str] = self.cache.get(vid1, set())
		c2:set[str] = self.cache.get(vid2, set())
		return (vid1 in c2) or (vid2 in c1) or (not c1.isdisjoint(c2))

	def addAsCompared(self, vdata1, vdata2):
		vid1 = vdata1['entity']['uid']
		vid2 = vdata2['entity']['uid']
		c1 = self._get_data_cached(vid1)
		c2 = self._get_data_cached(vid2)
		c1.add(vid2)
		c2.add(vid1)

		if not vdata1.get('individual_rating', None):
			vdata1['individual_rating'] = dict()
		vdata1['individual_rating']['last_compared_at'] = datetime.datetime.utcnow().isoformat()
		vdata1['individual_rating']['n_comparisons'] = vdata1['individual_rating'].get('n_comparisons',0) + 1
		self.toupdate.add(vid1)

		if not vdata2.get('individual_rating', None):
			vdata2['individual_rating'] = dict()
		vdata2['individual_rating']['last_compared_at'] = datetime.datetime.utcnow().isoformat()
		vdata2['individual_rating']['n_comparisons'] = vdata2['individual_rating'].get('n_comparisons',0) + 1
		self.toupdate.add(vid2)

	def clear(self):
		self.cache.clear()

In [ ]:
# Phase Init (Create first comparison from rate_later & low_comparisons)
def phase_init(rate_later:list, low_comparisons:list, DCC: DistComparisonChecker):
	"""
	Get the latest compared video in low_comparisons, and compare it to next rate_later video
	"""
	vid_new = rate_later.pop(0)
	DCC._get_data_cached(vid_new['entity']['uid'])

	vid_old = None
	while not vid_old or DCC.are_sharing_comparisons(vid_new, vid_old):
		vid_old = max(
			[vdata for vdata in low_comparisons if not DCC.have_cached_comparisons_shared(vid_new, vdata)],
			key=lambda vdata:(
				-get(vdata, 0, 'individual_rating', 'n_comparisons'), # Fewer comparisons first
				get(vdata, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at') # Compared the more recently
				# Publication date older first (low_comparisons list order)
			)
		)
	low_comparisons.remove(vid_old)

	DCC.addAsCompared(vid_new, vid_old)
	ab = rndAB()
	return (vid_new, f'[1] https://tournesol.app/comparison?uid{ab[0]}=' + vid_new['entity']['uid'] + f"&uid{ab[1]}=" + vid_old['entity']['uid'])


In [ ]:
# Phase Expand (Create next comparison from rate_later)
def phase_expand(vid_old, rate_later:list, DCC: DistComparisonChecker):
	"""
	Take first 2 in RATE_LATER

	If ko, change the 2nd one with the next and retry. If no more next: END
	When ok, print comparison URL, pop the first one & push it to the end of COMPARED, then go to Phase 1
	"""
	i = 0
	while DCC.are_sharing_comparisons(vid_old, rate_later[i]):
		i+=1
		if i > len(rate_later):
			# Pick any if all have shared comparisons
			i=0
			break

	vid_new = rate_later.pop(i)

	DCC.addAsCompared(vid_new, vid_old)
	ab = rndAB()
	return (vid_new, f'[+] https://tournesol.app/comparison?uid{ab[0]}=' + vid_new['entity']['uid'] + f"&uid{ab[1]}=" + vid_old['entity']['uid'])


In [ ]:
# Phase Intricate (Create next comparison from low_comparisons)
def phase_intricate(vid_new, compared:list, DCC: DistComparisonChecker):
	"""
	Take first from rate later
	Take first from compared

	Check for both: https://api.tournesol.app/users/me/comparisons/videos/yt:<vid>/
	There should be NO vid in common in both lists of entities.

	If ko, take next one from Compared and retry. If no more next: END
	When ok, print comparison URL and pop the one from COMPARED (if still less than 4 cmps, push it to the end of COMPARED), then go Phase 2
	"""
	i=0
	while DCC.are_sharing_comparisons(vid_new, compared[i]):
		i += 1
		if i > len(compared):
			# Pick any if all have shared comparisons
			i = 0
			break

	vid_old = compared.pop(i)
	DCC.addAsCompared(vid_new, vid_old)
	ab = rndAB()
	return (vid_new, f'[x] https://tournesol.app/comparison?uid{ab[0]}=' + vid_old['entity']['uid'] + f"&uid{ab[1]}=" + vid_new['entity']['uid'])


In [ ]:
# Phase Consolidate (Create comparison for 2 high_comparisons videos)
def phase_consolidate(high_comparisons:list, DCC: DistComparisonChecker):
	"""
	From all compared videos, take only the ones that has:
		- been compared at least 3 time by me
		- been compared by at least 2 different contributors
		- not been compared in the last 6 days
	From these videos, get pairs such as videos in a pair has:
		- Same language
		- Exact same number of (individual) comparison made
		- No comparison in common (DCC)
	From all pairs, take the one having the smallest score, score being calculated by adding:
		+ Difference of individual score
		+ Difference of collective score
		+ Sqrt(Difference of duration)
		+ Sqrt(Difference of time between both video aired)
	
	Compare them (Do not remove them from the list)

	If none found, return None
	"""
	# Compute minmax & fast access to some data in vdata
	mins = dict()
	maxs = dict()
	vdata = []
	today = datetime.datetime.utcnow()
	for v in high_comparisons:
		lng:str = get(v, None, 'entity', 'metadata', 'language')
		cmp:int = get(v, 0, 'individual_rating', 'n_comparisons')
		cnt:int = get(v, 0, 'collective_rating', 'n_contributors')
		indiv_score:float = get_individual_score(v)
		coll_score:float = get(v, 0, 'collective_rating', 'tournesol_score')
		duration:int = math.sqrt(get(v, 0, 'entity', 'metadata', 'duration'))
		aired = math.sqrt( (today - dateparse.parse(get(v, None, 'entity', 'metadata', 'publication_date'), ignoretz=True)).days )
		last_cmp = (today - dateparse.parse(get(v, None, 'individual_rating', 'last_compared_at'), ignoretz=True)).days
		
		if indiv_score is not None and cnt >= 2 and cmp >= 3 and last_cmp > 6 and lng in LNGS:
			vdata.append({
				'lng': lng,
				'cmp': cmp,
				'ind': indiv_score,
				'col': coll_score,
				'dur': duration,
				'air': aired,
				'full': v
			})
			for (key,val) in (('ind', indiv_score), ('col', coll_score), ('dur', duration), ('air', aired)):
				if not key in mins:
					mins[key] = val
					maxs[key] = val
				elif val < mins[key]:
					mins[key] = val
				elif val > maxs[key]:
					maxs[key] = val

	# Find best pair
	bestpair:tuple[any,any] = None
	while not bestpair:
		bestfitness:float = 999999
		maxfitness:float = -999999
		for i1 in range(1,len(vdata)):
			v1 = vdata[i1]

			for v2 in vdata[0:i1]:
				if (v1['lng'] != v2['lng']
					or v1['cmp'] != v2['cmp']
					or DCC.have_cached_comparisons_shared(v1['full'], v2['full'])
				):
					continue

				# Get pair score
				fitness = (
					( (v1['ind'] - v2['ind'])/(maxs['ind']-mins['ind']) )**2 * 2 # Similar individual score (0 - 1)*2
					+ ( (v1['dur'] - v2['dur'])/(maxs['dur']-mins['dur']) )**2 * 1 # Similar video duration (0 - 1)*1
					+ ( (v1['air'] - v2['air'])/(maxs['air']-mins['air']) )**2 * 0.5 # Similar airing date (0 - 1)*0.5
					+ (v1['cmp']-4) * 0.02 # Prefer videos with fewer comparisons
				)

				if fitness < bestfitness:
					bestfitness = fitness
					bestpair = (v1, v2)
				elif fitness > maxfitness:
					maxfitness = fitness

		if not bestpair:
			return None

		if DCC.are_sharing_comparisons(bestpair[0]['full'], bestpair[1]['full']):
			bestpair = None

	DCC.addAsCompared(bestpair[0]['full'], bestpair[1]['full'])
	ab = rndAB()
	return (f'[*] https://tournesol.app/comparison?uid{ab[0]}=' + bestpair[0]['full']['entity']['uid'] + f"&uid{ab[1]}=" + bestpair[1]['full']['entity']['uid'] 
		 + f" {'{f='}{1-bestfitness/maxfitness:.2%}{'}'}"
		 + f" (ind: {bestpair[0]['ind']:.0f}/{bestpair[1]['ind']:.0f}, dur: {bestpair[0]['dur']:.0f}/{bestpair[1]['dur']:.0f}, air: {bestpair[0]['air']:.0f}/{bestpair[1]['air']:.0f})"
	)


In [ ]:
# Comparisons Generator
def getComparisons(rate_later: list, low_comparisons: list, high_comparisons:list, DCC: DistComparisonChecker, ignore: set[str]):
	# Phase 0
	vid = None
	while True:
		(vid, cmp) = phase_init(rate_later, low_comparisons, DCC)
		yield cmp
		if vid['entity']['uid'] not in ignore:
			break
		else:
			print('##### Current vid is ignored #####')
			rate_later.remove(vid)

	while True:
		consolidated = phase_consolidate(high_comparisons, DCC)
		if consolidated:
			yield consolidated

		if get(vid, 0, 'individual_rating', 'n_comparisons') >= 2:
			(newvid, cmp) = phase_expand(vid, rate_later, DCC)
		else:
			(newvid, cmp) = phase_intricate(vid, low_comparisons, DCC)

		if not newvid:
			break
		yield cmp
		if newvid['entity']['uid'] not in ignore:
			vid = newvid
		else:
			print('##### Current vid is ignored #####')
			if vid in rate_later: rate_later.remove(vid)
			if vid in low_comparisons: low_comparisons.remove(vid)

	yield 'NO MORE'

# Main part

There are 3 cells bellow:

- "INIT/RESET": To run once to initialize the tool
- "IGNORE": To add videos to not be compared
- "CONTINUE": To run as many time as you want, to get more comparisons links generated

If at anytime you do comparisons not suggested by the tool, plase run again INIT/RESET to synchronize the tool with your current tournesol account

In [ ]:
# INIT/RESET ORDO (Replay this cell everytime any comparison other than suggested by this notebook has been made)
(RATE_LATER, LOW_CMPS, HIGH_CMPS) = get_lists()
DCC = DistComparisonChecker()
ignore:set[str] = set()
comparison = getComparisons(RATE_LATER, LOW_CMPS, HIGH_CMPS, DCC, ignore)
comparisons = []
print('Initialized !\n')
print('[+]:', len(RATE_LATER))
print('[x]:', len(LOW_CMPS))
print('[*]:', len(HIGH_CMPS))

In [ ]:
# IGNORE: Add here videos to ignore if a suggestion told to compare a video you don't want to compare for now.
# You wan add more videos between ORDO steps, then continue ordo without RESET

# ignore.add('yt:abcdefghijk')
print('Ignored videos:', len(ignore))

In [ ]:
# CONTINUE ORDO (Replay this cell everytime to get next comparison)
# Print previous
if comparisons:
	print('Previous:')
	for (i,cmp) in enumerate(comparisons[-5:], start=max(1, len(comparisons)-4)):
		print(f'{i:4d}.', cmp)

# Print new
print()
comparisons.append(next(comparison))
print('\nNew comparison:')
print(f'{len(comparisons):4d}.', comparisons[-1])

In [ ]:
# All tournesol videos having 1 or 2 contributors and positive score
def public2contrib():
	print('Extracting recommendations...', end=' ')
	allRes:list = callTournesolMulti('polls/videos/recommendations/', 'exclude_compared_entities=false&unsafe=true&exclude_compared_entities=true', start=2000, limit=25000)
	allRes = [vdata for vdata in allRes if 
		    get(vdata, 0, 'collective_rating', 'n_contributors') in [1,2]
		and get(vdata, 0, 'collective_rating', 'n_comparisons') >= 3
		and get(vdata, 0, 'collective_rating', 'tournesol_score')/get(vdata, 0, 'collective_rating', 'n_contributors') >= 5
	]
	allRes.sort(key=lambda vdata: (
		round(get(vdata, 0, 'collective_rating', 'tournesol_score')),
		get(vdata, 0, 'collective_rating', 'n_comparisons')
	), reverse=True)
	authors:dict[str,list] = dict()
	for vdata in allRes:
		a = get(vdata, '???', 'entity', 'metadata', 'uploader')
		authors.setdefault(a, list()).append(vdata)

	print()
	print('# Tournesol missing contributors list')

	print('Extraction date:', str(datetime.datetime.now())[:16])
	print()
	print('Found', len(allRes), 'videos by', len(authors), 'channels with good score but less than 3 contributors')
	print()

	years={get(vdata, '???', 'entity', 'metadata', 'publication_date')[0:4] for vdata in allRes}

	for y in sorted(years, reverse=True):
		print(f'\n## Year {y}')
		for vdata in allRes:
			if get(vdata, '???', 'entity', 'metadata', 'publication_date')[0:4] == y:
				tscore=get(vdata, 0, 'collective_rating', 'tournesol_score')
				tcmps=get(vdata, 0, 'collective_rating', 'n_comparisons')
				tctrs=get(vdata, 0, 'collective_rating', 'n_contributors')
				tchannel=get(vdata, '???', 'entity', 'metadata', 'uploader').replace('\\', '\\\\').replace('(', '\(').replace(')', '\)').replace('[', '\[').replace(']', '\]')
				tname=get(vdata, '???', 'entity', 'metadata', 'name').replace('\\', '\\\\').replace('(', '\(').replace(')', '\)').replace('[', '\[').replace(']', '\]')
				print(f"- {tscore:3.0f}🌻 ({tcmps} cmps / {tctrs} user{'s' if tctrs > 1 else ''}) {tchannel}: [{tname}](https://tournesol.app/entities/{vdata['entity']['uid']})")

public2contrib()

In [ ]:
for vdata in RATE_LATER:
	print(f"[{get(vdata, '????-??-??', 'entity', 'metadata', 'publication_date')[:10]}, {get(vdata, 0, 'collective_rating', 'tournesol_score'):4.1f}, {get(vdata, 0, 'collective_rating', 'n_comparisons'):3d}/{get(vdata, 0, 'collective_rating', 'n_contributors')}, {get(vdata, 0, 'individual_rating', 'n_comparisons')}/me] {get(vdata, 'unknown', 'entity', 'metadata', 'uploader')} : {get(vdata, 'unknown', 'entity', 'metadata', 'name')}")
# print('[x]:', len(LOW_CMPS))
# print('[*]:', len(ALL_COMPARED))